# Assignment - 2: Data Representation and Point Cloud Operations

Team Name: Sudarshan_Rishabh \
Roll number:  2021701008 2021701030

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 26/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.

# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Introduction to types of Transformations and Homogeneous coordinates

In robotics applications, it is inevitable to keep track of the frames of multiple objects/worlds. These frames can be transformations from one coordinate frame to the other. **Homogeneous coordinates** help in keeping track of various coordinate frames and allow performing composition of various transforms. We will first try to understand between types of transformations and their invariant properties.
1. What is the difference between Affine, Similarity, and Euclidean transform? What are the invariant properities of each type of transform?\

Eucledian transform is subset of Similarity transform, and Similarity transform is subset of Affine transform.\
Eucledian transform:- Translation + Rotation. (Preserves Angles and Distances)\
Similarity transform:- Eucledian + Uniform Scaling. (Preserves Angles) \
Affine transform:- Similarity + Scaling + Reflection + Shear. (Preserves parallel lines)\
Invariant property:- lines and parallelism is preserved in all. Affine transformation does not necessarily preserve angles between lines or distances between points, though it does preserve ratios of distances between points lying on a straight line. \
Ref:- https://www.cs.tau.ac.il/~dcor/Graphics/cg-slides/trans3d.pdf \
Ref:- https://www.cs.toronto.edu/~kyros/courses/418/Lectures/lecture.2010f.02.pdf \



2. Watch this [video](https://www.youtube.com/watch?v=PvEl63t-opM) to briefly understand homogeneous coordinates. What are points at infinity? What type of transformation can you apply to transform a point from infinity to a point that is not at infinity? \

Answer: \
We use projective geometry to depict a 3-D scenes on 2-D surfaces.
In Eucledian geometry, parallel lines don’t meet in a point. But Projective geometry, all lines intersect. 
The points at infinity are the points which are added to the space to get the projective completion. 
Eg. Projection of two paralleel railway tracks on projective plane. Ideal points on the projective plane are located at infinity, and have coordinates of the form $(x_1, x_2, 0)$. 
\
https://pointatinfinityblog.wordpress.com/2016/04/11/points-at-infinity-i-projective-geometry/ \
https://www.ipb.uni-bonn.de/html/teaching/3dcs-ge-2021/stachniss/2020-3dcs-02-homogeneous-coords-4.pdf \
\
Projective transformations are more general than affine transformations because the fourth row does not have to contain 0, 0, 0 and 1. So, we can use Projective transformations for point at infinity.\
P is projective transfomation  and H is affine transformatio matrix. \
$$ P =\left[\begin{array}{rrrr}p_{11} & p_{12} &  p_{13} & p_{14} \\ p_{21} & p_{22} & p_{23} & p_{24} \\
    p_{31} & p_{32} & p_{33} & p_{34} \\
    p_{41} & p_{42} & p_{43} & p_{44}   \end{array}\right] 
$$ 
$$ H = \left[\begin{array}{rrrr}a_{11} & a_{12} &  a_{13} & a_{14} \\ a_{21} & a_{22} & a_{23} & a_{24} \\
    a_{31} & a_{32} & a_{33} & a_{34} \\
    0 & 0 & 0 & 1  \end{array}\right] 
$$ \
Ref: https://pages.mtu.edu/~shene/COURSES/cs3621/NOTES/geometry/geo-tran.html



3. Using homogeneous coordinates we can represent different types of transformation as point transforms vs. frame transforms. Concatenation of transforms (whether you post multiply transformation matrices or pre-multiply transformation matrices) depends on the problem and how you are viewing it. Try to understand the difference between frame vs. point transformations from this [video](https://youtu.be/Za7Sdegf8m8?t=1834). Let's assume that our camera and world frames are coinciding with each other. We need to estimate the camera to world **frame** transformation matrix after applying the transformations defined below in terms of $T_i$.We apply **frame** transform to move the camera in the world in the following order:
    1. $T_1$ from the camera coordinate frame.
    2. $T_2$ from the world coordinate frame.
    3. $T_3$ from the world coordinate frame.
    4. $T_4$ from the camera coordinate frame.
    5. $T_5$ from the camera coordinate frame.\

Answer: \
$(T_3 * T_2 * T_i * T_1 * T_4 * T_5)$


# Visualise the Data

Point clouds are a collection of points that represent a 3D shape or feature. Each point has its own set of X, Y and Z coordinates and in some cases additional attributes. A popular way to obtain this is by photogrammetry, though here we will use LiDAR data.

LiDAR is a remote sensing process which collects measurements used to create 3D models and maps of objects and environments. Using ultraviolet, visible, or near-infrared light, LiDAR gauges spatial relationships and shapes by measuring the time it takes for signals to bounce off objects and return to the scanner.

1. Download the data from [here](https://iiitaphyd-my.sharepoint.com/:f:/g/personal/venkata_surya_students_iiit_ac_in/EnYAMaTVIhJItzKYqtahE30BRKB6p6UfHN3TyJzvo6Mw0g?e=PegWds). It contains the LIDAR sensor output and odometry information per frame.

    The .bin files contain the 3D point cloud captured by the LIDAR in this format - x, y, z, and reflectance. 

    The odometry information is given in the `odometry.txt` file, which is a 12 element vector. Reshape each of the first 77 rows to a 3x4 matrix to obtain the pose.
    

2. Obtain the point cloud from this and visualise for 1-2 frames.

In [1]:
# 1a .Reading the point cloud and making it into n*3 matrix

import numpy as np
import open3d as o3d
import os

fileDir = os.path.dirname(os.path.realpath('__file__'))
path_to_pcd = os.path.join(fileDir, 'data/LiDAR')

path_to_files = sorted(os.listdir(path_to_pcd)) 
# print( path_to_files)

for count, pcd_file in enumerate(path_to_files): 
#     print(count)
    complete_path = os.path.join( path_to_pcd , pcd_file )
    bin_pcd = np.fromfile(complete_path , dtype=np.float32) #single array
    points = bin_pcd.reshape((-1, 4))[:, 0:3]  #make an n*3 matrix and remove last column
#     print((points))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# 1b .Reading odometry.txt and converting into 3*4 matrix

import numpy as np
import open3d as o3d
import os
from numpy import genfromtxt

def load_poses(path):
    # odometry_data = np.loadtxt(path)
    odometry_data = genfromtxt(path, delimiter=',')
    return odometry_data

fileDir = os.path.dirname(os.path.realpath('__file__'))
odometry_file_path = os.path.join(fileDir, 'data/odometry.csv')

# odometry_file_path =  "/home/rishabh/notebook/MR2021-Assignment-2/data/odometry.csv"
odometry = load_poses( odometry_file_path)

# print(odom)

T_current_frame = np.zeros( (3, 4))
for count, odom in enumerate(odometry):
    T_current_frame[0:3 , 0:4] = np.reshape( odom , (3,4))
    
#     print(count)
#     print(T_current_frame)

    

In [3]:
# 2. Obtain the point cloud from this and visualise for 1-2 frames.

import numpy as np
import open3d as o3d
import os
from numpy import genfromtxt

fileDir = os.path.dirname(os.path.realpath('__file__'))
path_to_pcd = os.path.join(fileDir, 'data/LiDAR')
path_to_files = sorted(os.listdir(path_to_pcd)) 
# print( path_to_files)

vis = o3d.visualization.Visualizer()
vis.create_window()

for count, pcd_file in enumerate(path_to_files): 
    print("Count: ", count)
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    
    complete_path = os.path.join( path_to_pcd , pcd_file )
    bin_pcd = np.fromfile(complete_path , dtype=np.float32) #single array
    points = bin_pcd.reshape((-1, 4))[:, 0:3]  #make an 3*4 matrix and remove last column
    
    o3d_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))
    vis.add_geometry(o3d_pcd)
    vis.run()
    
    if count == 2:
        break

vis.destroy_window()

Count:  0
Count:  1
Count:  2


# Transform 

The point cloud obtained is with respect to the LiDAR frame. The poses however, are in the camera frame. If we want to combine the point clouds from various frames, we need to bring them to the camera frame. 

1. Refer to the image below and apply the required transformation to the point cloud. 

2. Then, register all point clouds into a common reference frame and visualise it (Open3D). It is helpful to use homogeneous coordinates to keep track of the different frames.

3. Write a function to transform the registered point cloud from the world to the $i^{th}$ camera frame, wherein $i$ is the input to the function.

4. \[Bonus\] Move around in the registered point cloud using arrow keys like you would do in a game. For this you will have to regularly transform the entire registered world to your current camera frame and visualize repeatedly. You may choose to avoid visualizing points that are behind the camera in this case as they are not visible from the scene. You may also visualize points at a max depth to make the process easier.

![](./img/transform.png)

In [4]:
#  1. apply the required transformation to the point cloud

import numpy as np
import open3d as o3d
import os
from numpy import genfromtxt

def load_poses(path):
    odometry_data = genfromtxt(path, delimiter = ',')
    return odometry_data

def eul_to_rotm_xyz(a,b,c):
    Ma = np.array([[1, 0, 0], [0, np.cos(c), -np.sin(c)], [0, np.sin(c), np.cos(c)]])
    Mb = np.array([[np.cos(b), 0, np.sin(b)], [0, 1, 0], [-np.sin(b), 0, np.cos(b)]])
    Mc = np.array([[np.cos(a), -np.sin(a), 0], [np.sin(a), np.cos(a), 0], [0, 0, 1]])
    M1 = np.matmul(Ma,Mb)
    M = np.matmul(M1,Mc)
    return M

fileDir = os.path.dirname(os.path.realpath('__file__'))
odometry_file_path = os.path.join(fileDir, 'data/odometry.csv')
odom = load_poses(odometry_file_path)

path_to_pcd = os.path.join(fileDir, 'data/LiDAR')
path_to_files = sorted(os.listdir(path_to_pcd)) 

R = eul_to_rotm_xyz(np.pi/2, 0, np.pi/2) # 3*3 matrix only rotation

for pcd_file in path_to_files:
    complete_path = os.path.join( path_to_pcd , pcd_file )
    bin_pcd = np.fromfile(complete_path , dtype=np.float32)
    
    points_current_frame = bin_pcd.reshape((-1, 4))[:, 0:3] # n*3 matrix
    points_current_frame = (R@(points_current_frame.T)).T
    
# points_current_frame are points observed in camera frame.
    

In [5]:
# 2. register all point clouds into a common reference frame and visualise it 

import numpy as np
import open3d as o3d
import os

def eul_to_rotm_xyz(a,b,c):
    Ma = np.array([[1, 0, 0], [0, np.cos(c), -np.sin(c)], [0, np.sin(c), np.cos(c)]])
    Mb = np.array([[np.cos(b), 0, np.sin(b)], [0, 1, 0], [-np.sin(b), 0, np.cos(b)]])
    Mc = np.array([[np.cos(a), -np.sin(a), 0], [np.sin(a), np.cos(a), 0], [0, 0, 1]])
    M1 = np.matmul(Ma,Mb)
    M = np.matmul(M1,Mc)
    return M

vis = o3d.visualization.Visualizer()
vis.create_window()
world_points = o3d.geometry.PointCloud()

pose_count = 9
all_points = np.array( [[ 0,0,0]])

T_world = np.array( [  [  1 ,0,0,0 ] , 
                    [0 ,1 ,0 ,0 ] , 
                    [0,0,1,0] , 
                    [0,0,0,1]  ])

T_current_frame = np.array( [  [  1 ,0,0,0 ] , 
                    [0 ,1 ,0 ,0 ] , 
                    [0,0,1,0] , 
                    [0,0,0,1]  ] , dtype ='float32')

iter_count = 0
for pcd_file in path_to_files:
    complete_path = os.path.join( path_to_pcd , pcd_file )
    bin_pcd = np.fromfile(complete_path , dtype=np.float32 )

    points_current_frame = bin_pcd.reshape((-1, 4))[:, 0:3]

    pcd2 = o3d.geometry.PointCloud()
    pcd2.points = o3d.utility.Vector3dVector(points_current_frame)
    dpcd = pcd2.voxel_down_sample(voxel_size = 0.5)

    points_current_frame = np.array(dpcd.points)

    R = eul_to_rotm_xyz(np.pi/2, 0, np.pi/2)
    R1 = np.eye(4)
    R1[:3, :3] = R
    points_current_frame = (R@(points_current_frame.T)).T

    one = np.ones( ( np.shape( points_current_frame)[0] ,1)  )
    points_current_frame = np.append( points_current_frame,  one , axis=1)

    T_current_frame[0:3 , 0:4] = np.reshape( odom[pose_count] , (3,4))
    update_points = ((T_current_frame)@points_current_frame.T).T

    all_points =np.append(all_points  , update_points[:,0:3] , axis= 0)
    pose_count +=1
    iter_count +=1 

world_points.points = o3d.utility.Vector3dVector(all_points)
world_points = world_points.voxel_down_sample(voxel_size = 0.05)
o3d.visualization.draw_geometries([world_points])

o3d.io.write_point_cloud('World_pts.pcd' , world_points)

vis.destroy_window()

[Open3D WARNING] GLFW Error: The GLFW library is not initialized


In [6]:
# 3. a function to transform the registered point cloud from the world to the ith camera frame

# RUN IN SEQUENCE

frame_index_i = int ( input(" Enter Frame index"))

camera_pose_at_frame_index_i = np.array( [  [  1 ,0,0,0 ] , 
					[0 ,1 ,0 ,0 ] , 
					[0,0,1,0] , 
					[0,0,0,1]  ] , dtype ='float32')

camera_pose_at_frame_index_i[0:3, 0:4] =  np.reshape( odom[frame_index_i] , (3,4))

one_arr =  np.ones( ( np.shape(all_points)[0] , 1 ) )
all_points_copy = np.append( all_points , one_arr , axis =1) 
all_points_at_frame_i = ((camera_pose_at_frame_index_i)@all_points_copy.T).T

reference_coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame( size=20, origin= [  camera_pose_at_frame_index_i[0][3] , camera_pose_at_frame_index_i[1][3] , camera_pose_at_frame_index_i[2][3] ])

world_points.points = o3d.utility.Vector3dVector(all_points_at_frame_i[:, 0:3])
o3d.visualization.draw_geometries([world_points , reference_coordinate])

vis.destroy_window()

 Enter Frame index3
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


In [8]:
# Bonus

import pygame
import sys
import numpy as np
import time
import numpy as np
import open3d as o3d
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pygame.init()
fps=7
fpsclock=pygame.time.Clock()
sur_obj=pygame.display.set_mode((400,300))
pygame.display.set_caption("Keyboard_Input")
White=(255,255,255)
p1=10
p2=10
step=5
opp = 1
x_mov,y_mov,z_mov = 1,1,1 
R = np.eye(3)
x_total, y_total, z_total, yaw = 0,0,0,0
theta = 0

vis = o3d.visualization.Visualizer()
vis.create_window()

complete_path = 'World_pts.pcd'
global pcd
pcd = o3d.io.read_point_cloud(complete_path)
pcd = pcd.voxel_down_sample(voxel_size = 0.1)

all_pts = np.array( pcd.points)

one = np.ones( ( np.shape( all_pts)[0] ,1)  )
all_pts = np.append( all_pts,  one , axis=1)

H = np.array( [  [  1 ,0,0,0 ] , 
                    [0 ,1 ,0 ,0 ] , 
                    [0,0,1,0] , 
                    [0,0,0,1]  ] , dtype ='float32')

global world_refrence_frame 
world_refrence_frame= o3d.geometry.TriangleMesh.create_coordinate_frame( size=20, origin= [  0,0,0 ])
global reference_coordinate 
reference_coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame( size=20, origin= [  0,0,0 ])

global all_pts_point_cloud 
all_pts_point_cloud = o3d.geometry.PointCloud()
all_pts_point_cloud = pcd 

def perform_transformation(R,tx,ty,tz):
    global reference_coordinate 
    global pcd
    reference_coordinate = reference_coordinate.translate((tx, ty , tz))

    for i in range(5):
        reference_coordinate.rotate(R)
        vis.update_geometry(reference_coordinate)
        vis.poll_events()
        vis.update_renderer()

    H[0:3, 0:3] = R
    H[0][3] = tx
    H[1][3] =ty 
    H[2][3] = tz

    pts = (H@(all_pts.T)).T 
    all_pts_point_cloud.points = o3d.utility.Vector3dVector(pts[: , 0:3]) 
    vis.update_geometry(all_pts_point_cloud)
    vis.poll_events()
    vis.update_renderer()
vis.add_geometry(all_pts_point_cloud)
# vis.update_geometry(pcd)
vis.poll_events()
vis.update_renderer()

while True:
    sur_obj.fill(White)
    pygame.draw.rect(sur_obj, (255,0,0), (p1, p2, 70, 65))
    for eve in pygame.event.get():
        if eve.type==pygame.QUIT:
            pygame.quit()
            sys.exit()

    key_input = pygame.key.get_pressed()   
    if key_input[pygame.K_LEFT]:
        p1 -= step
        print("LEFT")
        vis.add_geometry(reference_coordinate)
        vis.add_geometry(world_refrence_frame)
        perform_transformation(R,0,-1,0)

    if key_input[pygame.K_RIGHT]:
        p1 += step
        print("RIGHT")
        vis.add_geometry(reference_coordinate)
        vis.add_geometry(world_refrence_frame)
        perform_transformation(R,0,1,0)
    
    if key_input[pygame.K_UP]:
        p2 -= step
        print("FORWARD")
        x_mov = 1 
        x_total = x_total + x_mov #in world_frame
        vis.add_geometry(reference_coordinate)
        vis.add_geometry(world_refrence_frame)
        perform_transformation(R,1,0,0)

    if key_input[pygame.K_DOWN]:
        p2 += step
        print("BACKWARD")
        vis.add_geometry(reference_coordinate)
        vis.add_geometry(world_refrence_frame)
        perform_transformation(R,-1,0,0)
    
    if key_input[pygame.K_KP9]: #keypad 9
        print("YAW +")
        yaw =  1  #degree
        theta = np.deg2rad(yaw)
        R = np.array( [[np.cos(theta),-np.sin(theta),         0], 
              [np.sin(theta),np.cos(theta), 0], 
              [   0,            0,          1]])
        vis.add_geometry(reference_coordinate)
        vis.add_geometry(world_refrence_frame)
        perform_transformation(R,x_total,y_total,z_total)

    if key_input[pygame.K_KP7]: #keypad 7
        print("YAW -")
        yaw =  -1  #degree
        theta = np.deg2rad(yaw)
        R = np.array( [[np.cos(theta),-np.sin(theta),         0], 
              [np.sin(theta),np.cos(theta), 0], 
              [   0,            0,          1]])

        vis.add_geometry(reference_coordinate)
        vis.add_geometry(world_refrence_frame)
        perform_transformation(R,x_total,y_total,z_total)
        continue
    
    if key_input[pygame.K_KP8]: #keypad 8
        
        print("Roll - ")
        roll = -1 
        roll = np.deg2rad(roll)
        R = np.array( [ [ np.cos(roll) ,  0,   np.sin(roll) ] , 
                      [0,1,0] , 
                     [ -np.sin(roll) ,  0 , np.cos(roll)   ]  ] )
        vis.add_geometry(reference_coordinate)
        vis.add_geometry(world_refrence_frame)
        perform_transformation(R,x_total,y_total,z_total)
        continue 

    if key_input[pygame.K_KP0]: #keypad 0
        
        print("Pitch - ")
        pitch = -1 
        pitch = np.deg2rad(pitch)
        R = np.array( [ [ np.cos(pitch) ,  -np.sin(pitch) ,  0 ] , 
                      [np.sin(pitch),np.cos(pitch),0] , 
                     [ 0 ,  0 , 1  ]  ] )
        vis.add_geometry(reference_coordinate)
        vis.add_geometry(world_refrence_frame)
        perform_transformation(R,x_total,y_total,z_total)
        continue    
        
    if key_input[pygame.K_KP5]: #keypad 5
        p2 -= step
        print("UP")
        vis.add_geometry(reference_coordinate)
        vis.add_geometry(world_refrence_frame)
        perform_transformation(R,0,0,1)
     
    if key_input[pygame.K_KP2]: #keypad 2
        p2 -= step
        print("DOWN")
        vis.add_geometry(reference_coordinate)
        vis.add_geometry(world_refrence_frame)
        perform_transformation(R,0,0,-1)

    pygame.display.update()
    fpsclock.tick(fps)
    vis.add_geometry(world_refrence_frame)

vis.destroy_window()

pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
RIGHT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
LEFT
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
FORWARD
Roll - 
Roll - 
YAW +
Pitch - 
DOWN
DOWN
DOWN
UP
UP
UP
Roll - 
Roll - 
YAW -
YAW -


KeyboardInterrupt: 

## Occupancy Map

Occupancy grid maps are discrete fine grain grid maps. These maps can be either 2-D or 3-D. Each cell in the occupancy grid map contains information on the physical objects present in the corresponding space. Since these maps shed light on what parts of the environment are occupied, and what is not, they are really useful for path planning and navigation.

Occupancy grid maps are probabilistic in nature due to noisy measurements. Each cell can have three states: Occupied, unoccupied, and unknown. For the purpose of this assignment, you can ignore the unknown and work in a binary setting where 1 is occupied and 0 is unoccupied.

1. The task here is to create an occupancy map for each LiDAR scan. You do not need to apply bayesian update rules here, just keep it simple. 

2. Now, using the *registered* point cloud, generate occupancy maps for each frame. What difference do you expect to see between the two methods?

You can mark a cell as occupied based on a threshold of how many different z values are there for a particular (x,y) cell.

In [7]:
# 1. create an occupancy map for each LiDAR scan.

import numpy as np
import open3d as o3d
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy import genfromtxt

def load_poses(path):
    odometry_data = genfromtxt(path, delimiter = ',')
    return odometry_data


def get_limits_of_map(coordinates):
    X = coordinates[:, 0 ]
    Y = coordinates[: ,1]
    Z = coordinates[: ,2]
    xmin = np.amin(X)
    xmax = np.amax(X)
    ymin = np.amin(Y)
    ymax = np.amax(Y)
    return xmin , ymin , xmax ,ymax

def set_grid_resolution_and_sub_resolution(xmin , ymin , xmax, ymax ):
    pts = 1000
    sub_pts = 3
    xres =  (xmax -xmin)/pts
    yres = ( ymax - ymin)/pts 
    res = [ xres , yres]
    return res , sub_pts

def get_point_set( Q , points_current_frame, threshold):
    vaild_set = np.array([0 , 0 , 0])
    for pts in points_current_frame:
        # print(Q)
        # print(pts)
        L1 = np.sum(np.abs( Q - pts[0:2]))
        # print(L1)
        z_count = 0
        if( L1 < threshold/2 ):
            print(pts)
            if(pts[2] > -1.0):
                z_count +=1 
            if(z_count > 5):
                return True 

            
fileDir = os.path.dirname(os.path.realpath('__file__'))
odometry_file_path = os.path.join(fileDir, 'data/odometry.csv')
odom = load_poses(odometry_file_path)

path_to_pcd = os.path.join(fileDir, 'data/LiDAR')
path_to_files = sorted(os.listdir(path_to_pcd)) 

complete_path = os.path.join( path_to_pcd , path_to_files[0] )
bin_pcd = np.fromfile(complete_path , dtype=np.float32)
points_current_frame = bin_pcd.reshape((-1, 4))[:, 0:3]
xmin , ymin , xmax, ymax =   get_limits_of_map( points_current_frame)
res, sub_res = set_grid_resolution_and_sub_resolution(xmin , ymin , xmax, ymax )
height_counter = np.zeros( ( 1001 , 1001 ))
Occ_map = np.zeros( ( 1001 , 1001 ))
height_counter = np.zeros( ( 1001 , 1001 ))
Occ_map = np.zeros( ( 1001 , 1001 ))

for pt in points_current_frame:
    x = int( (pt[0] -xmin )/ res[0]) 
    y = int( (pt[1] -ymin)/ res[1])
    if( pt[2] > -1):
        height_counter[x][y] +=1
    if( height_counter[x][y] > 3): # If the number of Z values is greater than 3 then consider obstacle 
        Occ_map[x][y] = 255 ## white color represents an obstacle 

# print(Occ_map)
# plt.imshow(Occ_map)
# plt.show()
cv2.imshow('Map', height_counter)
cv2.waitKey(0)

-1

In [8]:
# 2. using the registered point cloud, generate occupancy maps for each frame

import numpy as np
import open3d as o3d
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

complete_path = 'World_pts.pcd'

pcd = o3d.io.read_point_cloud(complete_path)
points_current_frame = np.asarray( pcd.points)
xmin , ymin , xmax, ymax =   get_limits_of_map( points_current_frame)
res, sub_res = set_grid_resolution_and_sub_resolution(xmin , ymin , xmax, ymax )
height_counter = np.zeros( ( 1001 , 1001 ))
Occ_map = np.zeros( ( 1001 , 1001 ))

for pt in points_current_frame:
    x = int( (pt[0] - xmin ) / res[0])
    y = int( (pt[1] -ymin)  / res[1])
    if( pt[2] > -1):
        height_counter[x][y] +=1
    if( height_counter[x][y] > 3):
        Occ_map[x][y] = 255

# print(Occ_map)
cv2.imshow('Map', Occ_map)
cv2.waitKey(0)

-1

The second method has a large number of points and is a dense map, whereas the first method is shown only for frame one has few points and is less dense. 